In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

22/07/24 00:04:08 WARN Utils: Your hostname, kuchoco-GP62-6QE resolves to a loopback address: 127.0.1.1; using 192.168.219.113 instead (on interface wlp2s0)
22/07/24 00:04:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/24 00:04:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']
df_green = spark.read.parquet('data/origin_green/').select(columns)

In [9]:
duration_rdd = df_green.rdd
duration_rdd.take(3)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2019, 12, 19, 0, 52, 30), PULocationID=264, DOLocationID=264, trip_distance=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 1, 9, 45, 58), PULocationID=66, DOLocationID=65, trip_distance=1.28),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 1, 9, 41, 38), PULocationID=181, DOLocationID=228, trip_distance=2.47)]

In [23]:
import pandas as pd
rows = duration_rdd.take(10)
pd.DataFrame(rows, columns=columns)

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2,2019-12-19 00:52:30,264,264,0.00
1,2,2020-01-01 09:45:58,66,65,1.28
2,2,2020-01-01 09:41:38,181,228,2.47
3,1,2020-01-01 09:52:46,129,263,6.30
4,1,2020-01-01 09:19:57,210,150,2.30
5,1,2020-01-01 09:52:33,35,39,3.00
6,2,2020-01-01 09:10:18,25,61,2.77
7,2,2020-01-01 10:03:14,225,89,4.98
8,2,2020-01-01 09:04:11,129,129,0.71
9,2,2020-01-01 09:25:52,129,83,0.80


In [21]:
def apply_batch_count(partition):
    cnt = 0
    for row in partition:
        cnt += 1
    return [cnt]

def apply_batch_count_pandas(rows):
    df = pd.DataFrame(rows, columns=columns)
    cnt = len(df)
    return [cnt]

In [22]:
print(duration_rdd.mapPartitions(apply_batch_count).collect()) # 6개의 파티션이 있음
print(duration_rdd.mapPartitions(apply_batch_count_pandas).collect()) 

[447770, 398632, 318616, 176592, 164193, 135368, 93005]


[447770, 398632, 318616, 176592, 164193, 135368, 93005]


In [24]:
#  model = ...

def model_predict(df):
    # y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row
        

In [26]:
duration_rdd.mapPartitions(apply_model_in_batch).toDF().show()

+-----+--------+--------------------+------------+------------+-------------+------------------+
|Index|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+-----+--------+--------------------+------------+------------+-------------+------------------+
|    0|       2|                  []|         264|         264|          0.0|               0.0|
|    1|       2|                  []|          66|          65|         1.28|               6.4|
|    2|       2|                  []|         181|         228|         2.47|12.350000000000001|
|    3|       1|                  []|         129|         263|          6.3|              31.5|
|    4|       1|                  []|         210|         150|          2.3|              11.5|
|    5|       1|                  []|          35|          39|          3.0|              15.0|
|    6|       2|                  []|          25|          61|         2.77|             13.85|
|    7|       2|              